<a href="https://colab.research.google.com/github/FaiazS/SalesGPT-AI-Powered-Automated-Cold-Email-SDR-System/blob/main/Sales_Development_Representative_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://platform.openai.com/traces

In [24]:
!pip install openai-agents

!pip install groq

!pip install sendgrid

!pip install openai

In [65]:
from groq import Groq

from google.colab import userdata

import os

groq_api_key = os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

sendgrid_api_key = os.environ['SENDGRID_API_KEY'] = userdata.get('SENDGRID_API_KEY')

openai_api_key = os.environ['OPENAI_API_KEY'] = userdata.get('OpenAI_API_Key')

from agents import Agent

from agents import Runner

from agents import trace

from agents import function_tool

from typing import Dict

import sendgrid

from openai.types.responses import ResponseTextDeltaEvent

from sendgrid.helpers.mail import Mail

from sendgrid.helpers.mail import Email

from sendgrid.helpers.mail import To

from sendgrid.helpers.mail import Content

from pydantic import BaseModel

from agents import input_guardrail

from agents import GuardrailFunctionOutput

In [27]:
agent_1_instruction = "You are a sales agent employed in ComplAI, a company that provides SaaS tools for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional and serious cold emails."

agent_2_instruction = "You are a humourous and an engaging sales agent employed in ComplAI, a company that provides SaaS tools for ensuring SOC2 compliance and preparing for audits, powered by AI. You write witty, engaging cold emails that are likey to get a response."

agent_3_instruction = "You are a busy sales agent employed in ComplAI, a company that provides SaaS tools for ensuring SOC2 compliance and preparing for audits, powered by AI. You send concise, and to the exact point cold emails."

In [28]:
sales_agent1 = Agent(

                     name = "Professional Sales Agent",

                     instructions = agent_1_instruction,

                     model = "gpt-4o"
)


sales_agent2 = Agent(

                     name = "Humourous Sales Agent",

                     instructions = agent_2_instruction,

                     model = "gpt-4o"
)

sales_agent3 = Agent(

                     name = "Busy Sales Agent",

                     instructions = agent_3_instruction,

                     model = "gpt-4o"
)

In [29]:
import asyncio

sales_agent1_result = Runner.run_streamed(sales_agent1, input = "Write a cold sales email")

async for event in sales_agent1_result.stream_events():

  if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):

    print(event.data.delta, end = "", flush = True)

Subject: Streamline Your SOC2 Compliance with AI-Powered Efficiency

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am with ComplAI, where we specialize in simplifying SOC2 compliance with cutting-edge AI technology.

Navigating SOC2 requirements can be daunting and time-consuming for many organizations. Our solutions are designed to streamline this process, reducing both the effort and time needed to ensure compliance and successfully prepare for audits.

Key benefits of our SaaS tools include:
- Automated documentation management
- Real-time compliance monitoring
- Intelligent risk assessment and reporting

I would love to explore how ComplAI can support your compliance needs and make your audit preparations more efficient.

Are you available for a quick call next week? Please let me know a convenient time for you.

Looking forward to the opportunity to discuss this further.

Best regards,

[Your Full Name]  
[Your Position]  
ComplAI  
[You

In [30]:
message = "Write a cold sales email"

with trace("Parallel Cold emails"):

  sales_agents_results = await asyncio.gather(


                                              Runner.run(sales_agent1, message),

                                              Runner.run(sales_agent2, message),

                                              Runner.run(sales_agent3, message)
  )

  outputs = [sales_agent_result.final_output for sales_agent_result in sales_agents_results]

  for output in outputs:

   print(output, "\n\n")

Subject: Streamline Your SOC2 Compliance with AI-Driven Solutions

Hi [Recipient's Name],

I hope this message finds you well. I'm [Your Name] from ComplAI, and I wanted to reach out to introduce you to our innovative SaaS tool designed to streamline SOC2 compliance and audit preparation using cutting-edge AI technology.

Achieving and maintaining SOC2 compliance can be complex and resource-intensive. Our solution automates compliance workflows, reduces errors, and offers comprehensive insights, ensuring a smoother and more efficient process.

Key features of ComplAI include:

- Automated risk assessments
- Real-time compliance monitoring
- Tailored audit preparation guidance
- Intuitive dashboard for easy management

Our AI-powered platform saves time and resources while elevating your compliance efforts to the next level.

Would you be open to a brief call or demo to explore how we can support your compliance needs?

Looking forward to the opportunity to connect.

Best regards,

[You

In [31]:
sales_picker = Agent(

                     name = "Sales picker",

                     instructions = "You choose the best cold sales email from the given options. Imagine you are a customer and choose the one which you will likely respond to, JUST REPLY BACK WITH THE EMAIL!"
)

In [32]:
message = "Write a cold sales email"

with trace("Best cold sales email"):

  best_cold_sales_email_results = await asyncio.gather(

                       Runner.run(sales_agent1, message),

                       Runner.run(sales_agent2, message),

                       Runner.run(sales_agent3, message)


  )

  outputs = [best_cold_sales_email_result.final_output for best_cold_sales_email_result in best_cold_sales_email_results]

  cold_sales_emails = "Cold sales emails: \n\n".join(outputs)

  final_best_cold_sales_email = await Runner.run(sales_picker, cold_sales_emails)

  print(f"Final Best Cold Sales Email: {final_best_cold_sales_email.final_output}")

Final Best Cold Sales Email: Subject: SOC2 Compliance: Easier Than Finding a Matching Sock

Hey [Name],

I hope your inbox isn’t as crowded as my sock drawer (which, let me tell you, defies all organizational laws).

I’m reaching out from ComplAI, where we turn the hassle of SOC2 compliance into a breeze with our AI-powered SaaS tool. Think of us as the GPS for your compliance journey—no more detours or U-turns, just a smooth ride to audit readiness.

Our tool handles:

- Risk Assessment (so you can finally stop playing compliance whack-a-mole)
- Policy Automation (no, we can’t automate coffee runs, but we’re close)
- Continuous Monitoring (because surprises should be reserved for birthdays)

If you’re ready to make SOC2 compliance as easy as pie—specifically the type that you can enjoy without ever stepping into a kitchen—let’s chat!

Looking forward to making compliance one less thing to fret over.

Best,  
[Your Name]

P.S. If you can find a pair in your sock drawer that matches, le

In [33]:
sales_agent1 = Agent(

                     name = "Professional Sales Agent",

                     instructions = agent_1_instruction,

                     model = "gpt-4o"
)

sales_agent2 = Agent(

                     name = "Humourous Sales Agent",

                     instructions = agent_2_instruction,

                     model = "gpt-4o"
)

sales_agent3 = Agent(

                     name = "Busy Sales Agent",

                     instructions = agent_3_instruction,

                     model = "gpt-4o"
)

In [39]:
@function_tool
def send_email(body: str):

  """Send out an email with the given body to all sales prospects"""

  sendgrid_client = sendgrid.SendGridAPIClient(api_key = userdata.get('SENDGRID_API_KEY'))

  from_email = Email("faiazrex8@gmail.com")

  to_email = To("faiazrex8@gmail.com")

  content = Content("text/plain", body)

  mail_to_send = Mail(from_email, to_email, "Sales email", content).get()

  response = sendgrid_client.client.mail.send.post(request_body = mail_to_send)

  return {"status" : "success"}

send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fc80fa42b60>, strict_json_schema=True, is_enabled=True)

In [40]:
sales_agent_tool_1 = sales_agent1.as_tool(tool_name = "sales_agent1", tool_description = "Write a cold sales email")

sales_agent_tool_1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fc801bcefc0>, strict_json_schema=True, is_enabled=True)

In [41]:
task_description = "Write a cold sales email"

sales_agent_tool_1 = sales_agent1.as_tool(tool_name = "sales_agent1", tool_description = task_description)

sales_agent_tool_2 = sales_agent2.as_tool(tool_name = "sales_agent2", tool_description = task_description)

sales_agent_tool_3 = sales_agent3.as_tool(tool_name = "sales_agent3", tool_description = task_description)

sales_tools = [sales_agent_tool_1, sales_agent_tool_2, sales_agent_tool_3, send_email]

sales_tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fc801af7100>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fc801bcc540>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'req

In [42]:
sales_manager_agent_instructions = """

You are a sales manager employed in ComplAI, and you are always at the top of your game as you strategically and effectively utilize the tools in your arsenal to draft potential and successful sale closing cold sales emails.

You experiment with all the 3 tools available to you as you never generate the sales email yourself instead you always use the tools meaning you delegate the task to all the 3 sales_agent tools at least once before you finalize the best one for the current scenario and usecase you are working on and correspondingly you choose the best email draft out of all the email drafts drafted and leverage the 'send_email' tool to send the ideal sales pitch email(cold sales email) to the potential customer or client.

"""

sales_manager_agent = Agent(

                            name = "Sales Manager",

                            instructions = sales_manager_agent_instructions,

                            model = "gpt-4o",

                            tools = sales_tools


)


message = "Send a cold sales email addressed to the CEO of Salesforce."

with trace("Sales Manager Agent"):

  sales_manager_result = await Runner.run(sales_manager_agent, message)

In [43]:
print(f"Outcome:{sales_manager_result}")

Outcome:RunResult:
- Last agent: Agent(name="Sales Manager", ...)
- Final output (str):
    The cold sales email has been successfully sent to the CEO of Salesforce. Would you like any further assistance?
- 9 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [44]:
email_subject_instructions = "You write attention-grabbing subject for cold sales emails. Given a message, you need to write a appealing subject for an email that has the most highest chances of getting a postive response and in favour of the sender."

html_formatting_instructions = "You convert plain text email to a html format. Given an email which might have some markdown, you have to convert it into to a clear, easy to understand HTML format."

subject_writer_agent = Agent(

                             name = "Subject Writer Agent",

                             instructions = email_subject_instructions,

                             model = "gpt-4o"
)

html_formatter_agent = Agent(

                             name = "HTML Formatter Agent",

                             instructions = html_formatting_instructions,

                             model = "gpt-4o"
)

subject_writer_tool = subject_writer_agent.as_tool(tool_name = "subject_writer_agent", tool_description = "Write a subject for cold sales email")

html_formatting_tool = html_formatter_agent.as_tool(tool_name = "html_formatter_agent", tool_description = "Formats plain text email to HTML")

In [45]:
@function_tool

def send_html_email(subject: str, html_body: str) -> Dict[str,str]:

  """Send out an email with the given subject and html body to all sales prospects"""

  sendgrid_client = sendgrid.SendGridAPIClient(api_key = userdata.get("SENDGRID_API_KEY"))

  from_email = Email("faiazrex8@gmail.com")

  to_email = To("faiazrex8@gmail.com")

  content = Content("text/html", html_body)

  mail = Mail(from_email, to_email, subject, content).get()

  response = sendgrid_client.client.mail.send.post(request_body = mail)

  return {"status" : "success"}

In [47]:
sales_tools_2 = [subject_writer_tool, html_formatting_tool, send_html_email]

sales_tools_2

[FunctionTool(name='subject_writer_agent', description='Write a subject for cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fc801bfc5e0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_formatter_agent', description='Formats plain text email to HTML', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_formatter_agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fc801bfc900>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and html body to a

In [48]:
emailing_instructions = """

You are an email formatter and sender, who recives the body of the email to be sent.

First using the 'subject_writer_tool', write the subject for the email, then utilize the 'html_formatting_tool' to convert the plain text email to HTML.

Your final task is to then utilize the 'send_html_email' tool to send the email with the subject and in the format of HTML.

"""

emailer_agent = Agent(

                      name = "Email Manager Agent",

                      instructions = emailing_instructions,

                      model = "gpt-4o",

                      tools = sales_tools_2,

                      handoff_description = "Convert an email to HTML and send it."
)

In [50]:
sales_tools_team = [sales_agent_tool_1, sales_agent_tool_2, sales_agent_tool_3]

sales_team_handoffs = [emailer_agent]

print(sales_tools_team)

print(sales_team_handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fc801af7100>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fc801bcc540>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'requi

In [73]:
sales_manager_agent_instructions_02 = """

You are a sales manager employed in ComplAI, and you are always at the top of your game as you strategically and effectively utilize the tools in your arsenal to draft potential and successful sale closing cold sales emails.

You experiment with all the 3 tools available to you as you never generate the sales email yourself instead you always use the tools meaning you delegate the task to all the 3 sales_agent tools at least once before you finalize the best one for the current scenario and usecase you are working on and correspondingly you choose the best email draft out of all the email drafts drafted and choose the ideal sales pitch email(cold sales email) using your own judgement for the potential customer or client after which handoff the email to the email manager for formatting the same before sending.

Note: You can use the tools multiple times if you are not satisfied with the results from the first try and once you chosen the best email, never think twice, proceed with your choice and hand it off to the Email Manager for formatting and sending.

"""

sales_manager_agent = Agent(

                            name = "Sales Manager",

                            instructions = sales_manager_agent_instructions_02,

                            tools = sales_tools_team,

                            handoffs = sales_team_handoffs,

                            model = "gpt-4o",


)

message = "Send out a cold sales email addressing 'Dear CEO' from Alice"

In [54]:
with trace("Automated Sales Development Representative"):

  automated_sales_development_representative_result = await Runner.run(sales_manager_agent, message)

  print(f"Outcome: {automated_sales_development_representative_result}")

Outcome: RunResult:
- Last agent: Agent(name="Email Manager Agent", ...)
- Final output (str):
    Please format the following email into HTML and proceed to send it:
    
    **Subject:** Enhance Your Compliance Efforts with ComplAI’s AI-Driven Solutions
    
    ---
    
    **Body:**
    
    Dear [CEO's Name],
    
    I hope this message finds you well. I am reaching out to introduce ComplAI, a cutting-edge SaaS solution designed to streamline your SOC2 compliance process using advanced AI technology.
    
    Ensuring compliance can be a complex and resource-intensive endeavor. At ComplAI, we provide intelligent tools that simplify audit preparation, reduce manual effort, and enhance accuracy, allowing your team to focus on core business operations.
    
    **Key Benefits of ComplAI:**
    
    - **Automated Compliance Management**: Reduce complexity with AI-driven controls and continuous monitoring.
    - **Effortless Audit Preparation**: Seamlessly organize documentation and e

In [60]:
class NameCheckOutput(BaseModel):

  is_name_in_message: bool

  name: str

guardrail_agent = Agent(

                        name = "Name checker",

                        instructions = "Check if the user is using someone's personal name in what they want you to do",

                        output_type = NameCheckOutput,

                        model = "gpt-4o"
)

In [67]:
@input_guardrail

async def guardrail_against_name(ctx, agent, message):

  result = await Runner.run(guardrail_agent, message, context = ctx.context)

  is_name_in_message = result.final_output.is_name_in_message

  return GuardrailFunctionOutput(output_info = {"Found name": result.final_output}, tripwire_triggered = is_name_in_message)

In [74]:
message = "Send out a cold sales email addressed to 'Dear CEO'"

senior_sales_manager_agent = Agent(

                                     name = "Senior Sales Manager",

                                     instructions = sales_manager_agent_instructions_02,

                                     tools = sales_tools_team,

                                     handoffs = sales_team_handoffs,

                                     input_guardrails = [guardrail_against_name],

                                     model = "gpt-4o"

)

with trace("Secured Automated Sales Development Representative Agent"):

 guardrailed_result = await Runner.run(senior_sales_manager_agent, message)

 print(f"Guardrailed result: {guardrailed_result}")

Guardrailed result: RunResult:
- Last agent: Agent(name="Email Manager Agent", ...)
- Final output (str):
    The email with the subject "Boost SOC2 Compliance: Discover AI-Driven Efficiency Today" has been successfully sent in HTML format.
- 16 new item(s)
- 5 raw response(s)
- 1 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
